# Projet 2 - Préparez des données pour un organisme de santé publique

In [2]:
import pandas as pd

chunk_size = 50000
batch_no = 1

dtype_cols = {
    0: 'object', 
    9: 'object', 
    15: 'object', 
    21: 'object', 
    22: 'object', 
    23: 'object', 
    24: 'object', 
    25: 'object', 
    29: 'object', 
    30: 'object', 
    31: 'object', 
    32: 'object', 
    33: 'object', 
    34: 'object', 
    42: 'object', 
    43: 'object', 
    44: 'object', 
    45: 'object', 
    46: 'object', 
    48: 'object', 
    49: 'object', 
    50: 'object', 
    51: 'object', 
    54: 'object', 
    65: 'object', 
    70: 'object', 
    148: 'object'
}

for chunk in pd.read_csv('data/en.openfoodfacts.org.products.csv', sep='\t', chunksize=chunk_size, dtype=dtype_cols):
    chunk.to_csv(f'data/chunks/en.openfoodfacts.org.products_{batch_no:03d}.csv', index=False)
    batch_no += 1

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7676\3152989140.py:9: DtypeWarning: Columns (65) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('data/en.openfoodfacts.org.products.csv', sep='\t', chunksize=chunk_size, dtype=dtype_cols):
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7676\3152989140.py:9: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('data/en.openfoodfacts.org.products.csv', sep='\t', chunksize=chunk_size, dtype=dtype_cols):
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7676\3152989140.py:9: DtypeWarning: Columns (65) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv('data/en.openfoodfacts.org.products.csv', sep='\t', chunksize=chunk_size, dtype=dtype_cols):
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7676\3152989140.py:9: DtypeWarning: Columns (51,65) have mixed 

In [3]:
# import pandas as pd

# lignes = pd.read_csv('data/en.openfoodfacts.org.products.csv', sep='\t', nrows=100 ) # , low_memory=False
# lignes.info()
# lignes.tail(10)